In [4]:
import numpy as np
import scipy as sp
import pandas as pd
import timeit
import datetime
import time
import pprint
import itertools
import pickle
import sklearn
import dask.dataframe as dd
import dask.array as da
import os
os.chdir('/mnt/t48/bighomes-active/sfeng/patentdiffusion/')
import fastparquet
seed = 3
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
# Distances
import scipy.spatial.distance as distance
# KL
from scipy.stats import entropy
# Normalize
from sklearn.preprocessing import normalize
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
# Pairwise distances
from sklearn.metrics.pairwise import pairwise_distances
import h5py
import logging

/homedir/eco/sfeng/bigdata/python/miniconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [9]:
ts = pd.read_pickle("DataStore/2018-07-P2/targ_samp_0717.pkl")

In [10]:
print(len(ts))

INFO:root:282079


In [11]:
logger = logging.getLogger()
logger.setLevel(logging.INFO)
logging.basicConfig(level=logging.INFO)
logger.addHandler(logging.FileHandler('Logs/repsim_{0}.log'.format(datetime.datetime.now().\
                                                            strftime("%Y-%m-%d"), 'a')))
print = logging.info
print('good day to you madam fiona')
print('started')
print(datetime.datetime.now())

INFO:root:good day to you madam fiona
INFO:root:started
INFO:root:2018-07-17 19:33:50.758988


In [13]:
import scipy.spatial.distance as distance
dms = ["ldavecs", "docvecs"]
# dms = ["ldavecs"]

l2 = ts.copy()
try:
    res = {}
    for dm in dms:
        print("Loading matrix and dict")
        print(datetime.datetime.now())
        pat_dict = fastparquet.ParquetFile("RawData/Cleaned/patabs7615_us_no_dup.parq").to_pandas(["patent"])["patent"].tolist()
        pat_dict = dict(zip(pat_dict, range(len(pat_dict))))
        # Store as dask array
        if dm == "ldavecs":
            ncols = 60
        else:
            ncols = 100
        pm = dd.read_parquet("DataStore/2018-07-P2/ML/{0}_pats_0712.parq".format(dm)).values.compute()
        
        
        for c in ['cite_msa_match_5', 'cite_msa_diff_5', 'cite_msa_match_10',
       'cite_msa_diff_10']:
            # Get sample; drop any that are missing
            print(len(l2))
            l3 = ts[["tp", "cp", c]].dropna(how="any")
            # Drop tp, cp not in pat_dict
            l3 = l3.loc[l3["tp"].isin(pat_dict.keys()) & l3["cp"].isin(pat_dict.keys())]
            print(len(l3))

            # Get patent vectors of each column
            tp_pv = [pm[pat_dict[p]].reshape(1,-1) for p in l3["tp"].tolist()]
            cp_pv = [pm[pat_dict[p]].reshape(1,-1) for p in l3["cp"].tolist()]
            mm = [pm[[pat_dict[q] for q in p if q in pat_dict.keys()]] for p in l3[c]]

            tmm = [1-np.mean(distance.cdist(tp_pv[i],mm[i], metric = "cosine")) for i in range(len(l3))]
            cmm = [1-np.mean(distance.cdist(cp_pv[i],mm[i], metric = "cosine")) for i in range(len(l3))]

            l3["sim_{0}_tp_{1}".format(dm, c)] = tmm
            l3["sim_{0}_cp_{1}".format(dm, c)] = cmm

            l2 = l2.merge(l3[["tp", "cp", "sim_{0}_tp_{1}".format(dm, c),\
                             "sim_{0}_cp_{1}".format(dm, c)]], how = "left", on = ["tp", "cp"])
        
except Exception as e:
    logging.exception("message")

INFO:root:Loading matrix and dict
INFO:root:2018-07-17 20:28:10.845913
INFO:root:282079
INFO:root:48540
/homedir/eco/sfeng/bigdata/python/miniconda3/lib/python3.6/site-packages/numpy/core/fromnumeric.py:2957: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/homedir/eco/sfeng/bigdata/python/miniconda3/lib/python3.6/site-packages/numpy/core/_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
INFO:root:282079
INFO:root:209891
INFO:root:282079
INFO:root:73345
INFO:root:282079
INFO:root:269958
INFO:root:Loading matrix and dict
INFO:root:2018-07-17 20:29:05.212607
INFO:root:282079
INFO:root:48540
INFO:root:282079
INFO:root:209891
INFO:root:282079
INFO:root:73345
INFO:root:282079
INFO:root:269958


In [14]:
l2 = l2.drop(['cite_msa_match_5', 'cite_msa_diff_5', 'cite_msa_match_10',
       'cite_msa_diff_10'],1)
l2.to_pickle("DataStore/2018-07-P2/targ_samp_cite_sim_0717.pkl")

In [15]:
np.round(l2.describe(),2)

,tp,cp,tp_primclass,tp_gyear,cp_gyear,tp_match_5,cp_match_5,tp_match_10,cp_match_10,sim_ldavecs_tp_cite_msa_match_5,...,sim_ldavecs_tp_cite_msa_diff_10,sim_ldavecs_cp_cite_msa_diff_10,sim_docvecs_tp_cite_msa_match_5,sim_docvecs_cp_cite_msa_match_5,sim_docvecs_tp_cite_msa_diff_5,sim_docvecs_cp_cite_msa_diff_5,sim_docvecs_tp_cite_msa_match_10,sim_docvecs_cp_cite_msa_match_10,sim_docvecs_tp_cite_msa_diff_10,sim_docvecs_cp_cite_msa_diff_10
count,282079.00,282079.00,282079.00,282079.00,282079.00,224116.00,180033.00,282079.00,225791.00,41180.00,...,249829.00,249829.00,41180.00,41180.00,193157.00,193157.00,63103.00,63103.00,249829.00,249829.00
mean,6114535.56,6115541.32,371.24,1998.26,1998.26,0.12,0.05,0.11,0.05,0.57,...,0.50,0.36,0.34,0.19,0.28,0.18,0.32,0.19,0.28,0.18
std,1300275.65,1300423.93,197.65,10.09,10.09,0.28,0.17,0.25,0.16,0.23,...,0.20,0.21,0.16,0.13,0.12,0.12,0.15,0.13,0.12,0.11
min,3930273.00,3930284.00,1.00,1976.00,1976.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,-0.43,-0.60,-0.49,-0.59,-0.55,-0.60,-0.52,-0.65
25%,5048077.50,5049108.50,224.00,1991.00,1991.00,0.00,0.00,0.00,0.00,0.40,...,0.36,0.19,0.23,0.10,0.21,0.10,0.22,0.10,0.21,0.11
50%,6016946.00,6017980.00,370.00,2000.00,2000.00,0.00,0.00,0.00,0.00,0.58,...,0.50,0.33,0.33,0.18,0.28,0.18,0.31,0.18,0.28,0.18
75%,7118685.00,7119716.00,482.00,2006.00,2006.00,0.00,0.00,0.04,0.00,0.75,...,0.64,0.50,0.43,0.27,0.36,0.25,0.41,0.26,0.35,0.25
max,9046924.00,9159107.00,850.00,2015.00,2015.00,1.00,1.00,1.00,1.00,1.00,...,1.00,1.00,0.94,0.85,0.93,0.86,0.94,0.85,0.93,0.86


In [13]:
l2.columns

Index(['tp', 'cp', 'tp_primclass', 'tp_gyear', 'tp_inv_msa', 'cp_gyear',
       'cp_inv_msa', 'tp_match_5', 'cp_match_5', 'tp_match_10', 'cp_match_10',
       'sim_ldavecs_tp_cite_msa_match_5', 'sim_ldavecs_cp_cite_msa_match_5',
       'sim_ldavecs_tp_cite_msa_diff_5', 'sim_ldavecs_cp_cite_msa_diff_5',
       'sim_ldavecs_tp_cite_msa_match_10', 'sim_ldavecs_cp_cite_msa_match_10',
       'sim_ldavecs_tp_cite_msa_diff_10', 'sim_ldavecs_cp_cite_msa_diff_10'],
      dtype='object')

_____
## Create regression data

In this version, just normalize, don't scale

In [5]:
l2 = pd.read_pickle("DataStore/2018-07-P2/targ_samp_cite_sim_0717.pkl")

In [6]:
p1 = l2[["tp", "tp_primclass", "tp_gyear", "tp_match_5", "tp_match_10", 'sim_ldavecs_tp_cite_msa_match_5',
        'sim_ldavecs_tp_cite_msa_match_10', 'sim_docvecs_tp_cite_msa_match_5', 'sim_docvecs_tp_cite_msa_match_10']]
p1.columns = ["patent", "tp_primclass","tp_gyear", "perc_match_5", "perc_match_10", 'sim_ldavecs_cite_msa_match_5',
         'sim_ldavecs_cite_msa_match_10','sim_docvecs_cite_msa_match_5', 'sim_docvecs_cite_msa_match_10']
p1["inv_msa_match"] = 1

p2 = l2[["cp", "tp_primclass", "tp_gyear", "cp_match_5", "cp_match_10", 'sim_ldavecs_cp_cite_msa_match_5',
        'sim_ldavecs_cp_cite_msa_match_10', 'sim_docvecs_cp_cite_msa_match_5', 'sim_docvecs_cp_cite_msa_match_10']]
p2.columns = ["patent", "tp_primclass","tp_gyear", "perc_match_5", "perc_match_10", 'sim_ldavecs_cite_msa_match_5',
         'sim_ldavecs_cite_msa_match_10','sim_docvecs_cite_msa_match_5', 'sim_docvecs_cite_msa_match_10']
p2["inv_msa_match"] = 0

p1 = p1.append(p2, ignore_index = True).reset_index(drop=True)

# eps = 0.01
# dv_min = 0.7
# def scale_docvecs(x):
#     scaled = ((x+dv_min)/(1+dv_min))*(1-eps)+eps
#     return scaled
# def scale_ldavecs(x):
#     scaled = x*(1-eps)+eps
#     return scaled

# sim_cols = [c for c in p1.columns if "sim_" in c]
# for c in sim_cols:
#     if "docvecs" in c:
#         p1[c] = p1[c].apply(scale_docvecs)
#     else:
#         p1[c] = p1[c].apply(scale_ldavecs)

# def scale_perc_match(x, eps):
#     scaled = x*(1-eps)+eps
#     return scaled

# Scale percentage citation MSA match to be in 0.0001,1 also: Definitely can't be 0.01!!!
# for c in ["perc_match_5", "perc_match_10"]:
#     p1[c] = p1[c].apply(lambda x: scale_perc_match(x, 0.0000008))
    
# Normalize these values too:
for c in ["perc_match_5", "perc_match_10", 'sim_ldavecs_cite_msa_match_5',
         'sim_ldavecs_cite_msa_match_10','sim_docvecs_cite_msa_match_5', 'sim_docvecs_cite_msa_match_10']:
    p1["norm_{0}".format(c)] = np.nan
    p1.loc[p1[c].notnull(), "norm_{0}".format(c)] = scaler.fit_transform(p1.loc[p1[c].notnull(), c].reshape(-1,1))
    

/homedir/eco/sfeng/bigdata/python/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/homedir/eco/sfeng/bigdata/python/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()
/homedir/eco/sfeng/bigdata/python/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:43: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.re

In [7]:
# Add year group
def get_year_group(x):
    if x in range(1975,1980):
        yg = "1975-80"
    elif x in range(1980,1985):
        yg = "1980-85"
    elif x in range(1985, 1990):
        yg = "1985-90"
    elif x in range(1990,1995):
        yg = "1990-95"
    elif x in range(1995,2000):
        yg = "1995-00"
    elif x in range(2000,2005):
        yg = "2000-05"
    elif x in range(2005,2010):
        yg = "2005-10"
    elif x in range(2010, 2015):
        yg = "2010-15"
    else:
        yg = np.nan
    return yg

def get_year_group_10(x):
    if x in range(1975,1985):
        yg = "1975-85"
    elif x in range(1985,1995):
        yg = "1985-95"
    elif x in range(1995, 2005):
        yg = "1995-05"
    elif x in range(2005,2015):
        yg = "2005-15"
    else:
        yg = np.nan
    return yg

p1["year_group"] = p1["tp_gyear"].apply(get_year_group_10)

In [8]:
pathdir = "DataStore/2018-07-P3/JTHReg0727/"
o_f = "targ_cite_sim_reg_0727.pkl"
p1.to_pickle(pathdir+o_f)

____

## Some plots and summaries

In [9]:
pathdir = "DataStore/2018-07-P3/JTHReg0727/"
o_f = "targ_cite_sim_reg_0727.pkl"
p1 = pd.read_pickle(pathdir+o_f)

In [11]:
np.round(p1["perc_match_10"].describe(),3)

count    507870.000
mean          0.082
std           0.217
min           0.000
25%           0.000
50%           0.000
75%           0.000
max           1.000
Name: perc_match_10, dtype: float64

In [26]:
p1.loc[p1["inv_msa_match"] == True, "perc_match_10"].describe()

count    282079.000000
mean          0.111751
std           0.249592
min           0.000100
25%           0.000100
50%           0.000100
75%           0.041763
max           1.000000
Name: perc_match_10, dtype: float64

In [27]:
p1.loc[p1["inv_msa_match"] == False, "perc_match_10"].describe()

count    225791.000000
mean          0.045340
std           0.158579
min           0.000100
25%           0.000100
50%           0.000100
75%           0.000100
max           1.000000
Name: perc_match_10, dtype: float64